Copyright 2024 Zhixuan Hu

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import numpy as np
import pandas as pd
import os, re, sys, json, time

In [2]:
sys.path.append(os.getcwd() + '\\..\\')

#### 为没有embedding的数据集标注embedding（text-embedding-3-small）

In [3]:
pattern = re.compile(r".*\.csv")
files = os.listdir('./')
files = [file for file in files if pattern.match(file)]
files

['chatgpt_HC3_all.csv',
 'gpt_0125_00.csv',
 'human_00.csv',
 'human_01_01.csv',
 'human_01_02.csv',
 'human_02.csv',
 'human_cook_01.csv',
 'human_cook_02.csv',
 'human_HC3_all.csv',
 'human_modern_01.csv',
 'human_modern_02.csv',
 'human_travel_01.csv',
 'human_travel_02.csv',
 'human_travel_03.csv',
 'questions_00.csv',
 'questions_01_01.csv',
 'questions_01_02.csv',
 'questions_02.csv',
 'questions_cook_01.csv',
 'questions_cook_02.csv',
 'questions_HC3_all.csv',
 'questions_modern_01.csv',
 'questions_modern_02.csv',
 'questions_travel_01.csv',
 'questions_travel_02.csv',
 'questions_travel_03.csv',
 'test_gpt_0125.csv',
 'test_human.csv',
 'test_questions.csv']

In [139]:
file = './chatgpt_HC3_all.csv'
df = pd.read_csv(file)
df

,Unnamed: 0,question_id,text,answer_id
0,0,3038,在世界历史中并没有出现过像海贼王中描述的那样的100年断层。但是，历史上有许多其他的断层事件...,1
1,1,7377,IPFilter 是一种网络过滤技术，它可以根据 IP 地址和端口号来控制网络流量。通常用于...,1
2,2,7997,我无法提供有关上海市位育中学建立新校舍以及入住时间的信息，因为我是一个大型语言模型，我的知识...,1
3,3,12092,抑郁症会导致人们对某些事情失去兴趣，包括出门和参加社交活动。如果您对人群感到恐惧，我建议您考...,1
4,4,8884,北海道大学的标志性建筑物包括：\n\n北海道大学主校区的主楼，该建筑是北海道大学的校园内最为...,1
...,...,...,...,...
17470,17470,435,我很抱歉，但我不知道您提到的丁晴手套是什么。可以提供更多信息吗？您可以提供这种手套的描述或者...,2
17471,17471,6352,电动助力转向系统（Electric Power Assisted Steering，简称EP...,1
17472,17472,934,下巴是一个人的面部特征之一，每个人都有自己独特的下巴，不同的下巴有不同的特点，也有不同的美感...,2
17473,17473,8521,附属中学是指在某个大学或其他高等学府下属的中学。这种学校通常是给大学的学生或工作人员的子女提...,3


In [140]:
model = 'text-embedding-3-small'
# 检查是否存在 'as' 列
if model not in df.columns:
    # 如果不存在，则创建一个空的字符串列
    df[model] = ''
df[model] = df[model].astype(str)

In [142]:
df_need = df[('"nan"' == df[model]) | (df[model]=='')]
# 防止过长超出token限制
tol = 5880
mask = df_need['text'].str.len() > tol
df_need.loc[mask, 'text'] = df_need.loc[mask, 'text'].str.slice(0, tol) #截断
df_need_list = df_need['text'].to_list()
df_need

,Unnamed: 0,question_id,text,answer_id,text-embedding-3-small
0,0,3038,在世界历史中并没有出现过像海贼王中描述的那样的100年断层。但是，历史上有许多其他的断层事件...,1,
1,1,7377,IPFilter 是一种网络过滤技术，它可以根据 IP 地址和端口号来控制网络流量。通常用于...,1,
2,2,7997,我无法提供有关上海市位育中学建立新校舍以及入住时间的信息，因为我是一个大型语言模型，我的知识...,1,
3,3,12092,抑郁症会导致人们对某些事情失去兴趣，包括出门和参加社交活动。如果您对人群感到恐惧，我建议您考...,1,
4,4,8884,北海道大学的标志性建筑物包括：\n\n北海道大学主校区的主楼，该建筑是北海道大学的校园内最为...,1,
...,...,...,...,...,...
17470,17470,435,我很抱歉，但我不知道您提到的丁晴手套是什么。可以提供更多信息吗？您可以提供这种手套的描述或者...,2,
17471,17471,6352,电动助力转向系统（Electric Power Assisted Steering，简称EP...,1,
17472,17472,934,下巴是一个人的面部特征之一，每个人都有自己独特的下巴，不同的下巴有不同的特点，也有不同的美感...,2,
17473,17473,8521,附属中学是指在某个大学或其他高等学府下属的中学。这种学校通常是给大学的学生或工作人员的子女提...,3,


#### 请求api

In [143]:
from embedding_models.text_embedding_3_small import get_embeddings
# 分批存储防止故障后丢失所有数据
num_chunk = 100
size_chunk = int(len(df_need) / num_chunk + 0.99)
indexer = df_need.index
for i in range(0, len(df_need), size_chunk):
    index = indexer[i:i+size_chunk]
    res = get_embeddings(df_need.loc[index, 'text'].to_list())
    if not res: res = ['nan'] * len(df_need.loc[index, 'text'])
    embedding_str = [json.dumps(embvec) for embvec in res]
    df_need.loc[index, 'text-embedding-3-small'] = embedding_str

2024-05-07 16:41:30,521 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:41:35,048 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:41:38,742 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:41:49,334 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:42:36,541 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:42:48,144 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:43:22,554 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:43:46,716 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07 16:43:59,215 - INFO - HTTP Request: POST https://api.crowtalk.live/v1/embeddings "HTTP/1.1 200 OK"
2024-05-07

In [144]:
df.loc[indexer] = df_need

In [145]:
time_str = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
os.rename(file, './history/' + file + '-' + time_str + '.csv')

df.to_csv(file, encoding='utf-8-sig')

#### 删除、放弃仍然nan的行

In [141]:
tol = 1
df = df[df['text'].fillna('').replace({'\n': ''}).apply(lambda x: len(x.replace('\n', '')) >= tol)]
df

,Unnamed: 0,question_id,text,answer_id,text-embedding-3-small
0,0,3038,在世界历史中并没有出现过像海贼王中描述的那样的100年断层。但是，历史上有许多其他的断层事件...,1,
1,1,7377,IPFilter 是一种网络过滤技术，它可以根据 IP 地址和端口号来控制网络流量。通常用于...,1,
2,2,7997,我无法提供有关上海市位育中学建立新校舍以及入住时间的信息，因为我是一个大型语言模型，我的知识...,1,
3,3,12092,抑郁症会导致人们对某些事情失去兴趣，包括出门和参加社交活动。如果您对人群感到恐惧，我建议您考...,1,
4,4,8884,北海道大学的标志性建筑物包括：\n\n北海道大学主校区的主楼，该建筑是北海道大学的校园内最为...,1,
...,...,...,...,...,...
17470,17470,435,我很抱歉，但我不知道您提到的丁晴手套是什么。可以提供更多信息吗？您可以提供这种手套的描述或者...,2,
17471,17471,6352,电动助力转向系统（Electric Power Assisted Steering，简称EP...,1,
17472,17472,934,下巴是一个人的面部特征之一，每个人都有自己独特的下巴，不同的下巴有不同的特点，也有不同的美感...,2,
17473,17473,8521,附属中学是指在某个大学或其他高等学府下属的中学。这种学校通常是给大学的学生或工作人员的子女提...,3,
